In [ ]:
import lemmy

# Create an instance of the standalone lemmatizer.
lemmatizer = lemmy.load("da")

corpus = """
                    Frederikshavn Kommune Ønsker du et varieret og spændende job inden for demensafsnittet, 
                     hvor alle bidrager aktivt i opgaveløsningen? Og har du lyst til at arbejde primært i fast dagvagt 
                     og have weekendvagt i ulige uger?  Vi søger en medarbejder med sundhedsfaglig baggrund til borgere med demens. 
                     Stillingen er på gennemsnitlig 30 timer pr. uge. Du kommer både selvstændigt og i samarbejde med 
                     andre samarbejdspartnere til, at arbejde med udgangspunkt i borgerens behov og værdier. 
                     Dette sker ud fra den rehabiliterende tilgang hvor du er med til at bevare/forbedre borgernes fysiske, 
                     psykiske og sociale funktioner. Det er således vigtigt, at det faldet dig naturligt at være fleksibel og
                     du er nærværende og tillidsskabende i relationen.
                     """

# Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
lemmatizer.lemmatize("", corpus)
print(len(corpus.strip()))
corpus.strip()

In [ ]:
from textblob import TextBlob
text = TextBlob(corpus.strip())
text.words
print(len(text.words))



In [ ]:
from nltk.stem.snowball import DanishStemmer

stemmer = DanishStemmer()

singles = [stemmer.stem(word) for word in text.words]

print(len(singles))
singles


In [ ]:
from nltk.corpus import stopwords
da_stop_words = stopwords.words('danish')

no_sw = [word for word in singles if not word in da_stop_words]

In [ ]:
print(len(no_sw))

no_sw

In [ ]:
text = " ".join(no_sw)
text

In [ ]:
lemmatized = lemmatizer.lemmatize("", text)

In [ ]:
final = TextBlob(lemmatized[0])

In [ ]:
len(final.words)

In [ ]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [5]:
%%time
import pandas as pd
import dask.dataframe as dd
import nltk
import numpy as np
from tqdm.notebook import tqdm
from pandarallel import pandarallel
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
import lemmy
import multiprocessing

from dask.diagnostics import ProgressBar
ProgressBar(minimum=5).register()


# pandarallel.initialize(progress_bar=True, use_memory_fs=None)
nltk.download('stopwords')

df = dd.read_csv('../data/interim/jobindex_123k.csv')
df = df.repartition(partition_size="100MB")
df_init = df

df_filtered = df_init[['title', 'company', 'description','location','link','ratings_link', 'source', 'date']]
df_filtered['merged'] = (
                        df_filtered['title'].fillna('') + ' '
                        + df_filtered['company'].fillna('') + ' '
                        + df_filtered['description'].fillna('')+ ' '
                        + df_filtered['location'].fillna('')+ ' '
#                         + df_filtered['link'].astype(str).fillna('')+ ' '
                        + df_filtered['ratings_link'].fillna('')+ ' '
                        + df_filtered['source'].fillna('')+ ' '
                        + df_filtered['date'].astype(str).fillna('')
                        )

# df['filtered'].compute()[0]

da_stop_words = stopwords.words('danish')
lemmatizer = lemmy.load("da")
stemmer = DanishStemmer()

all_descriptions = np.array(df_filtered['merged'])
all_descriptions_wo_stopwords = []


def pp_text(text):
    textblob = TextBlob(text.to_string(index=False))
    singles = [stemmer.stem(word) for word in textblob.words]
    no_stop_words = [word for word in singles if not word in da_stop_words]
    joined_text = " ".join(no_stop_words)
    final_text = lemmatizer.lemmatize("", joined_text)
    
    return final_text

# df_filtered['processed'] = df_filtered['merged'].map_partitions(pp_text, meta=df_filtered['merged']).compute()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pedromadruga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[########################################] | 100% Completed |  7.8s
[########################################] | 100% Completed |  8.2s
[########################################] | 100% Completed |  8.7s
[########################################] | 100% Completed |  9.2s
[########################################] | 100% Completed |  9.6s
[########################################] | 100% Completed | 13.1s
[########################################] | 100% Completed | 13.4s
[########################################] | 100% Completed | 13.6s
[########################################] | 100% Completed | 13.9s
[########################################] | 100% Completed | 14.3s
CPU times: user 24.1 s, sys: 1.45 s, total: 25.5 s
Wall time: 25 s


In [6]:
df_filtered['processed'] = df_filtered['merged'].map_partitions(pp_text, meta=df_filtered['merged']).compute()

[######################                  ] | 55% Completed |  6min 36.2s
[######################                  ] | 55% Completed |  6min 36.4s

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[######################                  ] | 55% Completed |  6min 38.0s


KeyboardInterrupt



[######################                  ] | 55% Completed |  6min 38.4s

In [ ]:
print(len(df_filtered))

print(df_filtered['processed'].compute()[0])



In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = TfidfVectorizer(stop_words="english", dtype="float32")
X = vectorizer.fit_transform(df_filtered['merged'].compute())

# print(df_filtered['merged'].compute())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_kernels
def search(job_title):
        
    stuff = [df_filtered.compute().index[df_filtered['merged'].str.contains(word, case=False).compute()].values for word in job_title.split()]
    return list(set(stuff[0]).intersection(*stuff))

search_results = search("jurist forsikring københavn")
print(f"number of direct results: {len(search_results)}")

# print(df['title'][first_result])

recommendations = []

# for result in search_results:
#     print(df['title'][result])


%time distances = pairwise_kernels(X, X[search_results[1]], n_jobs=-1)

for index, search_result in enumerate(search_results):
    distances = pairwise_kernels(X, X[search_results[index]], n_jobs=-1)
    for _index, value in enumerate(distances):
        recommendations.append({
            "title": df['title'][_index],
            "similarity": value[0],
            "location": df['location'][_index],
            "description": df['description'][_index]
        })

sorted_similarities = sorted(recommendations, key=lambda x: x['similarity'], reverse=True)

for index, result in enumerate(sorted_similarities):
    print(result)
    print('\n')
    if index > 20:
        break
    
 
# sorted_similarities = sorted(range(len(distances)), key=lambda k: distances[k], reverse=True)[:100]

# for index,value in tqdm(enumerate(sorted_similarities)):
#     print('\n')
#     print(df['title'][value])
#     print(df['description'][value])
#     print(df['location'][value])
#     print(distances[value])

In [ ]:
## word2vec

from dask.diagnostics import ProgressBar
ProgressBar().register()

def tokenize(text):
    stringified_text = text
#     print(type(stringified_text))
    tokenized_text = TextBlob(stringified_text)
    return tokenized_text.words
#     return text

# df_filtered['merged_tokenized'] = df_filtered['merged']

#     test_f, 
#     args=('col_1', 'col_2'), 
#     axis=1, 
#     meta=('result', int)

df_filtered["tokenized_merged"] = df_filtered.map_partitions(
    lambda df_filtered:\
    df_filtered.apply(\
    (lambda row: tokenize(row['merged'])), axis=1))\
    .compute()

df_filtered['tokenized_merged']




In [ ]:
%time df_filtered['stuff'].compute(scheduler='processes')
%time df_filtered['stuff'].compute()



In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# similarity = cosine_similarity(X)

def compute_cosine_distances(a, b):

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)

    return distance

# def convert_sparse_matrix_to_sparse_tensor(X):
#     coo = X.tocoo()
#     indices = np.mat([coo.row, coo.col]).transpose()
#     return tf.SparseTensor(indices, coo.data, coo.shape)

# ####################### ####################### ####################### 

# x_train = convert_sparse_matrix_to_sparse_tensor(X)

input_matrix = X.toarray()

# %time similarity_by_tf = compute_cosine_distances(input_matrix, input_matrix)
%time similarity = cosine_similarity(X)

# similarity_by_tf

# %time print(cosine_similarity(X))

In [ ]:
from dask_distance import cosine

print(X.shape)

distance_matrix = np.zeros(X.shape)

# print(X.T)

for i in X:
    for j in X.T:
        print(cosine(i,j))
        break


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df = pd.DataFrame(np.random.rand(1000,100))
df['distances'] = cosine_similarity(df, df.iloc[0:1]) # Here I assume that the parent vector is stored as the first row in the dataframe, but you could also store it separately

n = 10 # or however many you want
n_largest = df['distances'].nlargest(n + 1)

n_largest

In [ ]:
s = [2, 3, 1, 4, 5]
sorted(range(len(s)), key=lambda k: s[k], reverse=True)

In [4]:
import datetime
from dateutil import tz

HERE = tz.tzlocal()

print(HERE)

datetime.datetime(2020,3,1)

tzlocal()


datetime.datetime(2020, 3, 1, 0, 0)

In [7]:
import locale
import time

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
time.strptime('Dom, 01/02/1903', '%a, %d/%m/%Y')

time.struct_time(tm_year=1903, tm_mon=2, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=6, tm_yday=32, tm_isdst=-1)

In [14]:
import os
print(f'{os.path.abspath(os.getcwd())}/data/processed/{dataset_name}')

NameError: name 'dataset_name' is not defined

In [1]:
df_filtered.head()

NameError: name 'df_filtered' is not defined